In [ ]:
import scanpy as sc
import numpy as np
import os
import anndata2ri
import pathlib
import scvelo as scv
from scipy import io
import anndata#
import pandas as pd
from tqdm import tqdm
import argparse
import sys
import re
# Activate the anndata2ri conversion between SingleCellExperiment and AnnData
anndata2ri.activate()

#Loading the rpy2 extension enables cell magic to be used
#This runs R code in jupyter notebook cells
%load_ext rpy2.ipython

sc.settings.verbosity = 3
# sc.logging.print_versions()

import warnings
warnings.filterwarnings("ignore")

outdir = "/home/hieunguyen/CRC1382/outdir"
PROJECT = "SAlBounny_full"
path_to_loom_data = os.path.join(outdir, PROJECT, "velocyto_output")

#####------------------------------------------------------------#####
##### READ LOOM DATA

all_looms = [item for item in pathlib.Path(path_to_loom_data).glob("*.loom")]

velodata_dict = dict()
for input_loom in tqdm(all_looms):
    velodata = scv.read_loom(input_loom)
    if "200709" in input_loom.name:
        samplename = input_loom.name.replace("SAlBounny_200709_", "").replace(".loom", "")
    else:
        samplename = input_loom.name.replace("SAlBounny_", "").replace(".loom", "")

    new_obs = ["{}_{}_{}-1".format(samplename, samplename, item.split(":")[1].replace("x", "")) for item in velodata.obs.index]
    velodata.obs.index = new_obs
    velodata.obs["barcode"] = new_obs
    velodata.var_names_make_unique()
    pattern = re.compile('_[A|T|G|C]*-')
    new_obs = [samplename + pattern.search(string = item).group() + item.split("-")[1]
              for item in velodata.obs.index]
    velodata.obs.index = new_obs
    velodata_dict[samplename] = velodata
    velodata.var_names_make_unique()



  0%|          | 0/7 [00:03<?, ?it/s]


NameError: name 're' is not defined